## <span style="font-family:Georgia, serif;">**Twitter Sentiment Analysis** :Understanding Emotions in Tweets about Apple and Google products.</span>

![alt text](twits.jpg "Title")

## Overview

## Business Understanding

**Business Problem**: Using Sentiment Analysis to Improve Apple and Google Product Marketing Strategies 

The introduction of social media has completely changed how businesses interact with their consumers and the general public in today's connected society. While the digital age offers limitless possibilities for marketing and brand development, it also brings its own set of difficulties. One of these difficulties is the inability of enterprises to precisely gauge public opinion and feelings towards their goods or services.

In the age of social media, organizations are acutely aware of the need to harness the wealth of sentiment and emotion data available on these platforms. However, they often struggle to do so effectively, given the unprecedented speed, diversity, and complexity of social media communication. The dynamic nature of the medium, the diverse and contextual language used, the rapid increase of emojis and visual content, the volume of noise, and ethical concerns all contribute to the challenge of gauging public sentiment and emotions. 

To overcome these challenges, organizations must invest in advanced sentiment analysis tools and technologies, develop cultural and linguistic expertise, and strike a balance between data-driven insights and ethical considerations. By doing so, they can unlock the valuable insights hidden within the social media storm and use them to inform strategic decisions, enhance products and services, and build stronger connections with their audience in this rapidly evolving digital landscape.




## Data understanding

## Data Preparation

In [3]:
import pandas as pd
import re
import io
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

def remove_non_utf8(text):
    return re.sub(r'[^\x00-\x7F]+', '', text)

with open('data/judge_1377884607_tweet_product_company.csv', 'r', encoding='utf-8') as file:
    cleaned_text = remove_non_utf8(file.read())

df = pd.read_csv(io.StringIO(cleaned_text))
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [4]:
column_name_mapping = {'is_there_an_emotion_directed_at_a_brand_or_product': 'Sentiment'}
# Rename the columns using the .rename() method
df.rename(columns=column_name_mapping, inplace=True)


In [5]:
df['emotion_in_tweet_is_directed_at'].fillna('N/A', inplace=True)
df['tweet_text'].fillna('N/A', inplace=True)

In [6]:
def assign_brand(phrase):
    if 'iPad' in phrase or 'iPhone' in phrase :
        return 'Apple'
    elif 'Other Apple product or service' in phrase or 'Apple' in phrase:
        return 'Apple' 
    elif 'iPad or iPhone App' in phrase:
        return 'Apple'       
    elif 'Google' in phrase or 'Other Google product or service' in phrase:
        return 'Google'
    elif 'Android App' in phrase or 'Android' in phrase:
        return 'Android'
    else:
        return 'N/A'

df['brand'] = df['emotion_in_tweet_is_directed_at'].apply(assign_brand)

In [7]:
import nltk
import re
from nltk.tokenize import word_tokenize,TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
# nltk.download('punkt')
# nltk.download('stopwords')


def clean_and_preprocess_text(text):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    # Convert tokens to lowercase
    tokens = [token.lower() for token in tokens]
    # Remove mentions (words starting with '@') and URLs
    tokens = [token for token in tokens if not token.startswith('@') and not token.startswith('http')]
    # Remove punctuation and numbers using regular expressions
    tokens = [re.sub(r'[^a-zA-Z]', '', token) for token in tokens]
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # # Apply stemming using the Porter Stemmer
    # stemmer = PorterStemmer()
    # stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]

    cleaned_text = ' '.join(filtered_tokens) 
    return cleaned_text

In [8]:
df['tweet_text'][4]

"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)"

In [9]:
clean_and_preprocess_text(df['tweet_text'][4])

'great stuff fri sxsw  marissa mayer  google   tim oreilly  tech books  conferences   matt mullenweg  wordpress '

In [10]:
df['processed_text'] = df['tweet_text'].map(clean_and_preprocess_text)

In [11]:
# Define a mapping dictionary
sentiment_mapping = {'No emotion toward brand or product': 2.0,
                  'Positive emotion': 1.0, 
                  'Negative emotion': 0.0,
                  'I can\'t tell': 2.0}

# Use the .map() method to map values in column 'A' to new values
df['Sentiment'] = df['Sentiment'].map(sentiment_mapping)
df.sample(5)

,tweet_text,emotion_in_tweet_is_directed_at,Sentiment,brand,processed_text
333,I laughed at the iPad until I spent this week ...,iPad,1.0,Apple,laughed ipad spent week hauling macbook everyw...
3562,{link} Coinsidence? Sounds like a good strateg...,N/A,1.0,N/A,link coinsidence sounds like good strategy ...
2255,Stupid technology! You always fail at importan...,iPhone,0.0,Apple,stupid technology always fail important times...
2822,Fan up @mention To kick off #SXSWi @mention is...,N/A,2.0,N/A,fan kick sxswi giving away ipad visit fb pag...
5524,RT @mention Full #SXSW #touchingstories presen...,N/A,2.0,N/A,rt full sxsw touchingstories presentation link


In [12]:
from sklearn.model_selection import train_test_split
#creating new df where sentiment is either positive or negative
bi_tar = df[(df['Sentiment'] == 0)| (df['Sentiment'] == 1)]

X = bi_tar['processed_text']
y = bi_tar['Sentiment']

X_train_bi, X_test_bi, y_train_bi, y_test_bi = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
multi_tar = df.copy()
X = multi_tar['processed_text']
y = multi_tar['Sentiment']

y_dummies = pd.get_dummies(y)
X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(X, y_dummies, test_size=0.2, random_state=42)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

def csr_Tfid_vect(X_train,X_test):
    vectorizer = TfidfVectorizer()
    tf_idf_train = vectorizer.fit_transform(X_train)
    tf_idf_test = vectorizer.transform(X_test)

    tf_idf_train = csr_matrix(tf_idf_train)
    tf_idf_test = csr_matrix(tf_idf_test)

    return tf_idf_train,tf_idf_test

X_tf_idf_train_bi,X_tf_idf_test_bi = csr_Tfid_vect(X_train_bi,X_test_bi)

In [16]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Sequential
from keras.preprocessing import text, sequence

def token_seq(feature):
    tokenizer = text.Tokenizer(num_words=20000)
    tokenizer.fit_on_texts(list(feature))
    list_tokenized = tokenizer.texts_to_sequences(feature)
    seq = sequence.pad_sequences(list_tokenized, maxlen=100)
    return seq

## Text Analysis

In [27]:
from  nltk import FreqDist
import string

big_sentence = ' '.join(df['tweet_text'])
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
tweets_raw = nltk.regexp_tokenize(big_sentence, pattern)
tweets_raw = [word.lower() for word in tweets_raw]
stopwords_list = stopwords.words('english')
stopwords_list += list(string.punctuation)
tweets_raw_stopped = [word for word in tweets_raw if word not in stopwords_list]
tweets_freqdist = FreqDist(tweets_raw_stopped)
tweets_freqdist.most_common(10)

[('sxsw', 9256),
 ('mention', 6950),
 ('link', 4122),
 ('rt', 2959),
 ('ipad', 2882),
 ('google', 2373),
 ('apple', 2124),
 ('quot', 1621),
 ('iphone', 1525),
 ('store', 1420)]

In [18]:
total_word_count = sum(tweets_freqdist.values())
tweets_freqdist_top_10 = tweets_freqdist.most_common(10)
print(f'{"Word":10} Normalized Frequency')
for word in tweets_freqdist_top_10:
    normalized_frequency = word[1] / total_word_count
    print(f'{word[0]:10} {normalized_frequency:^20.4}')

Word       Normalized Frequency
sxsw              0.0858       
mention          0.06442       
link             0.03821       
rt               0.02743       
ipad             0.02671       
google            0.022        
apple            0.01969       
quot             0.01503       
iphone           0.01414       
store            0.01316       


In [19]:
from nltk.collocations import BigramCollocationFinder

bigram_measures = nltk.collocations.BigramAssocMeasures()
tweets_finder = BigramCollocationFinder.from_words(tweets_raw_stopped)
tweets_scored = tweets_finder.score_ngrams(bigram_measures.raw_freq)
tweets_scored[:15]

[(('rt', 'mention'), 0.02666802617674867),
 (('sxsw', 'link'), 0.008527835968929014),
 (('link', 'sxsw'), 0.007656513598190615),
 (('sxsw', 'rt'), 0.006275374946701025),
 (('mention', 'mention'), 0.005728481118258838),
 (('mention', 'sxsw'), 0.005478207671344618),
 (('apple', 'store'), 0.005348436254426132),
 (('sxsw', 'mention'), 0.004755195491370201),
 (('link', 'rt'), 0.004718117943679205),
 (('mention', 'google'), 0.004356611853691997),
 (('social', 'network'), 0.0040970690198550265),
 (('new', 'social'), 0.003781909864481563),
 (('mention', 'rt'), 0.0031886691014256317),
 (('network', 'called'), 0.003021820136816151),
 (('store', 'sxsw'), 0.003021820136816151)]

SXSW is best known for its conference and festivals that celebrate the convergence of tech, film, music, education, and culture.
RT is the first Russian 24/7 English-language news channel which brings the Russian view on global news.

In [29]:
from gensim.models import Word2Vec
from nltk import word_tokenize

data = df['processed_text'].map(word_tokenize)
model = Word2Vec(data, window=5, min_count=1, workers=4)
model.train(data, total_examples=model.corpus_count, epochs=10)

(724262, 978660)

In [40]:
wv = model.wv
wv.most_similar('sxsw')

[('shouldve', 0.6995207071304321),
 ('lousy', 0.6613374948501587),
 ('theinternet', 0.6485297679901123),
 ('spambots', 0.6440733671188354),
 ('rt', 0.6440219283103943),
 ('flu', 0.6315970420837402),
 ('kanyes', 0.629397451877594),
 ('ireport', 0.6291985511779785),
 ('trending', 0.6277326941490173),
 ('stpatrick', 0.6270562410354614)]

In [41]:
wv.most_similar(negative='sxsw')

[('worthy', 0.3314540982246399),
 ('ringing', 0.3195646405220032),
 ('mocking', 0.2917923629283905),
 ('rearfacing', 0.2840273976325989),
 ('participate', 0.27028223872184753),
 ('larry', 0.26822662353515625),
 ('arrive', 0.25811198353767395),
 ('hirer', 0.24964465200901031),
 ('whowillrise', 0.24164533615112305),
 ('linney', 0.24109816551208496)]

## Modeling & Evaluation

**Baseline Model**: Sentiment is either positive(1)or negative(0)


In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

rf =  Pipeline([('Random Forest', RandomForestClassifier(n_estimators=100, verbose=True))])
svc = Pipeline([('Support Vector Machine', SVC())])
lr = Pipeline([('Logistic Regression', LogisticRegression())])

models = [('Random Forest', rf),
          ('Support Vector Machine', svc),
          ('Logistic Regression', lr)]

scores = [(name, cross_val_score(model,X_tf_idf_train_bi, y_train_bi, cv=2).mean()) for name, model, in models]
scores 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    8.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    6.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


[('Random Forest', 0.8604913144444624),
 ('Support Vector Machine', 0.8568296515587877),
 ('Logistic Regression', 0.8443789251256308)]

In [21]:
X_t =  token_seq(X_train_bi) 

In [22]:
model_1 = Sequential()

model_1.add(Dense(units=64, input_shape=(100,)))
model_1.add(Dropout(0.5))

model_1.add(Dense(32, activation='relu'))
model_1.add(Dropout(0.5))

model_1.add(Dense(1, activation='sigmoid'))

model_1.compile(
    optimizer="adam",
    loss='binary_crossentropy',
    metrics=["accuracy"]
)

model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                6464      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 8577 (33.50 KB)
Trainable params: 8577 (33.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
model_1.fit(X_t, y_train_bi, epochs=15, batch_size=32, validation_split=0.1)

Epoch 1/15
77/77 [==============================] - 7s 20ms/step - loss: 165.1444 - accuracy: 0.6691 - val_loss: 78.4244 - val_accuracy: 0.8175
Epoch 2/15
77/77 [==============================] - 1s 10ms/step - loss: 98.2093 - accuracy: 0.6964 - val_loss: 28.1063 - val_accuracy: 0.7774
Epoch 3/15
77/77 [==============================] - 1s 10ms/step - loss: 60.1386 - accuracy: 0.6825 - val_loss: 15.5224 - val_accuracy: 0.8285
Epoch 4/15
77/77 [==============================] - 1s 10ms/step - loss: 39.4907 - accuracy: 0.6663 - val_loss: 8.9873 - val_accuracy: 0.8175
Epoch 5/15
77/77 [==============================] - 1s 10ms/step - loss: 23.8573 - accuracy: 0.6960 - val_loss: 2.8424 - val_accuracy: 0.8212
Epoch 6/15
77/77 [==============================] - 1s 11ms/step - loss: 12.7931 - accuracy: 0.7253 - val_loss: 1.0262 - val_accuracy: 0.8321
Epoch 7/15
77/77 [==============================] - 1s 12ms/step - loss: 10.8271 - accuracy: 0.7330 - val_loss: 0.5624 - val_accuracy: 0.8285
Ep

**Iterated Model**: Sentiment is either positive(1),negative(0),No emotion toward brand or product(2) or Not clear(3)

In [24]:
X_t_multi = token_seq(X_train_multi)

In [25]:
model_2 = Sequential()

model_2.add(Dense(64, activation='relu', input_shape=(100,)))
model_2.add(Dropout(0.5))

model_2.add(Dense(3, activation='softmax'))

model_2.compile(
    optimizer="adam",
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

model_2.fit(X_t_multi, y_train_multi, epochs=15, batch_size=32, validation_split=0.1)

Epoch 1/15
197/197 [==============================] - 6s 14ms/step - loss: 279.9000 - accuracy: 0.4511 - val_loss: 117.1826 - val_accuracy: 0.5688
Epoch 2/15
197/197 [==============================] - 2s 9ms/step - loss: 103.9855 - accuracy: 0.4712 - val_loss: 49.1618 - val_accuracy: 0.5086
Epoch 3/15
197/197 [==============================] - 2s 10ms/step - loss: 38.6627 - accuracy: 0.4661 - val_loss: 24.9363 - val_accuracy: 0.5458
Epoch 4/15
197/197 [==============================] - 2s 11ms/step - loss: 18.7626 - accuracy: 0.4774 - val_loss: 13.3960 - val_accuracy: 0.4871
Epoch 5/15
197/197 [==============================] - 2s 13ms/step - loss: 9.4230 - accuracy: 0.4810 - val_loss: 7.0505 - val_accuracy: 0.5287
Epoch 6/15
197/197 [==============================] - 2s 12ms/step - loss: 5.2740 - accuracy: 0.4978 - val_loss: 4.7952 - val_accuracy: 0.5287
Epoch 7/15
197/197 [==============================] - 2s 9ms/step - loss: 3.3290 - accuracy: 0.5287 - val_loss: 3.4064 - val_accurac

In [ ]:

from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np

# Load your GloVe model
glove_model = KeyedVectors.load_word2vec_format('path_to_glove_file', binary=False)

# Create document-level embeddings using GloVe
def document_vectorizer(text, model):
    words = text.split()
    vectors = [model[word] if word in model else np.zeros(model.vector_size) for word in words]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# Create GloVe-based embeddings for each text
document_embeddings = [document_vectorizer(text, glove_model) for text in X_train_multi]

# Convert embeddings to a numpy array
X = np.array(document_embeddings)

# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_train_multi, test_size=0.2, random_state=42)

# Train a classification model (e.g., Logistic Regression)

classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Evaluate your model on the test set
accuracy = classifier.score(X_test, y_test)
print(f'Accuracy: {accuracy}')


## Deployment